In [1]:
import pandas as pd
import numpy as np

# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) <br>Intro to Classification and Regression Trees (CARTs)
Week 6 | Day 2

### LEARNING OBJECTIVES
*After this lesson, you will be able to:*
- Describe what a decision tree is
- Explain how a classification tree works
- Explain how a regression tree works

## Recap

So far in this course, we've talked about regression and classification models.

In both cases, we used a **linear**, **parametric** model (with one exception).

What does that mean? And what was the exception?

## Linear

$ y = \beta_0 + \beta_1x_1...\beta_nx_n$


As the value of x increases, y increases in proportion to our $\beta$ parameter - The model is **linear in its parameters**.

## Parametric vs. Non-parametric Models


**Parametric models** have some fixed number of parameters that are solved for. There is an underlying assumption about the distribution from which the data is drawn.

**Non-parametric models** make no assumptions about the distribution of the underlying data. Additionally, rather than being though of as having no parameters, they can be though of as infitite in the number of parameters. The number of parameters can also vary with the size of the data.

## Intro to Decision Trees


### What a Decision Tree is
_Decision trees_ are a _non-parametric, hierarchical_ technique for both regression and classification.

**_Non-parametric_** means that the model is not described in terms of parameters like for example Logistic Regression. It also means that there is no underlying assumption on the distribution of data and of the error.

**_Hierarchical_** means that the model consists of a sequence of questions which yield a class label when applied to any record. In other words, once trained, the model behaves like a recipe, which will tell us a result if followed exactly.

## Decision Tree Example

Suppose I want to know which algorithm is best to use in a given scenario. I could build a hierarchical tree of decisions like this one which gives us a precise rule to make a decision depending on the values assumed by the features.

<img src="http://i.imgur.com/IaCnsUa.png" width=600>

## How that works

At each decision point, or node, in our tree, we need to choose either "yes" or "no" and move forward from there. This begins at our _root node_ and continues on through each _internal node_ until we reach a _leaf node_. At this leaf node, we have the answer to our problem.

## Let's look at another example of a decision tree classifier

<img src="http://i.imgur.com/IGebRXH.png" width=600>

Some questions to ponder:
- Why is credit history the first question?
- Why those specific income thresholds?
- Does the order we ask these question in matter?

## How to build a decision tree

In order to build a decision tree we need an algorithm that is capable of determining optimal choices for each node. One such algorithm is _Hunt's algorithm_: a _greedy recursive_ algorithm that leads to a _local optimum_.

- _greedy_: algorithm makes locally optimal decision at each step
- _recursive_: splits task into subtasks, solves each the same way
- _local optimum_: solution for a given neighborhood of points

The algorithm works by recursively partitioning records into smaller & smaller subsets. The partitioning decision is made at each node according to a metric called _purity_. A node is said to be 100% pure when all of its records belong to a single class (or have the same value).

## Example: binary classification with classes A, B

Given a set of records `Dt` at node `t`:

1. If all records in `Dt` belong exclusively to class A or exclusively to class B, then `t` is a _leaf node_ and no further iteration can take place
<br>
2. If however `Dt` contains records from both A and B, do the following:
    - create test condition to partition the records further
    - partition records in `r` to children according to test
    - `t` would then be internal node, with outgoing edges to child nodes

These steps are then recursively applied to each child node until they end in a leaf node (purely A or B)

## Types of splits

Splits can be 2 way or multi-way. Features can be categorical or continuous.

<img src="http://i.imgur.com/JOR2H8J.png" width=600>
<img src="http://i.imgur.com/KI8aBJX.png" width=600>

## Exercise

Fun time! You are all going to build a decision tree to classify candy! 

- Begin at the root node and at each point your classifier should ask one question and have a T or F answer
- True goes to the left, False goes to the right
- Continue until you have classified each candy to its name
- We will then test your models

## Optimization functions

Recall from the algorithm above, that we iteratively create test conditions to split the data. How do we determine the best split amongst all possible splits? Recall that no split is necessary (at a given node) when all records belong to the same class. Therefore we want each step to create the partition with the **highest possible purity**.

In order to do this, we will need an objective function to optimize. We want our objective function to measure the gain in purity from a particular split. Therefore we want it to depend on the class distribution over the nodes (before and after the split). For example, let $p(i|t)$ be the probability of class $i$ at node $t$ (eg, the fraction of records labeled $i$ at node $t$). Then for a binary (0/1) classification problem:

The maximum impurity partition is given by the distribution:
$$
p(0|t) = p(1|t) = 0.5
$$

where both classes are present in equal manner.

On the other hand, the minimum impurity partition is obtained when only one class is present, i.e:
$$
p(0|t) = 1 – p(1|t) = 1
$$

Therefore in the case of a binary classification we need to define an _impurity_ function that will smoothly vary between the two extreme cases of minimum impurity (one class or the other only) and the maximum impurity case of equal mix.

## How we measure "purity"

We can define several functions that satisfy these properties. Here are three common ones:


$$
\text{Entropy}(t) = - \sum_{i=0}^{c-1} p(i|t)log_2 p(i|t)
$$

$$
\text{Gini}(t) = 1 - \sum_{i=0}^{c-1} [p(i|t)]^2
$$

$$
\text{Classification error}(t) = 1 - \max_i[p(i|t)]
$$

<img src="http://i.imgur.com/UGPGzRq.png" width=600>

<center>Note that each measure achieves its max at 0.5, min at 0 & 1</center>

## One more step

Impurity measures put us on the right track, but on their own they are not enough to tell us how our split will do. We still need to look at impurity before & after the split. We can make this comparison using the gain:
$$
\Delta = I(\text{parent}) - \sum_{\text{children}}\frac{N_j}{N}I(\text{child}_j)
$$

Where $I$ is the impurity measure, $N_j$ denotes the number of records at child node $j$, and $N$ denotes the number of records at the parent node. When $I$ is the entropy, this quantity is called the _information gain_.

Generally speaking, a test condition with a high number of outcomes can lead to overfitting (ex: a split with one outcome per record). One way of dealing with this is to restrict the algorithm to binary splits only (CART). Another way is to use a splitting criterion which explicitly penalizes the number of outcomes (C4.5).

## Decision tree regression

In the case of regression, the outcome variable is not a category but a continuous value. We cannot therefore use the same measure of purity we used for classification. Instead we will use the following variation of the algorithm:

At each node calculate the variance of the data points at that node, then choose the split that generates the largest decrease in total variance for the children nodes. In other words we use variance as our measure of impurity and we want to maximize the decrease of total variance at each split.

In a regression tree the idea is this: since the target variable does not have classes, we fit a regression model to the target variable using each of the independent variables. Then for each independent variable, the data is split at several split points. At each split point, the "error" between the predicted value and the actual values is squared to get a "Sum of Squared Errors (SSE)". The split point errors across the variables are compared and the variable/point yielding the lowest SSE is chosen as the root node/split point. This process is recursively continued.

## Exercise

- Pair up and do a bit of research together on the pros and cons of decision trees

To think about:
- How do they stack up against other methods we've learned?
- Anything that is different about how you need to treat the data in a decision tree vs. other methods we've seen?
- What are some legitimate concerns about them?

- We'll discuss your findings together

## Overfitting

Check: What is overfitting?

## How to preventing overfitting with decision trees

In addition to determining splits, we also need a stopping criterion to tell us when we’re done. For example, we can stop when all records belong to the same class, or when all records have the same attributes. This is correct in principle, but would likely lead to overfitting. 

- _Pre-pruning_ - this involves setting a minimum threshold on the gain, and stopping when no split achieves a gain above this threshold. This prevents overfitting, but is difficult to calibrate in practice (may preserve bias!).
<br>
- _Post-pruning_ - this builds the full tree and then prunes as a post-processing step. To prune a tree, we examine the nodes from the bottom-up and simplify pieces of the tree (according to some criteria). Complicated subtrees can be replaced either with a single node, or with a simpler (child) subtree. The first approach is called subtree replacement, and the second is subtree raising.

<a name="ind-practice"></a>
## Independent Practice

1. Walk through the visual tutorial on R2D3 [Decision Trees](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/)
2. Find a decision tree model at [BigML](https://bigml.com/gallery/models) that you find interesting. Be prepared to talk about what the model is trying to predict and how it works.


## Conclusion
In this class we learned about Classification and Regression trees. We learned how Hunt's algorithm helps us recursively partition our data and how impurity gain is useful for determining optimal splits.
We've also reviewed pros/cons of decision trees and industry applications.
In the next class we will learn to use them in `Scikit-learn`.

### ADDITIONAL RESOURCES
- [Scikit-Learn Documentation](http://scikit-learn.org/stable/modules/tree.html)
- [Classification trees video](https://www.youtube.com/watch?v=p17C9q2M00Q)
- [Regression trees video](https://www.youtube.com/watch?v=zvUOpbgtW3c)
- [Decision trees on wikipedia](https://en.wikipedia.org/wiki/Decision_tree_learning)
- [Decision tree regression explained](http://www.saedsayad.com/decision_tree_reg.htm)